In [1]:
# https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning

In [2]:
# gemma-2 is available from transformers>=4.42.3
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [3]:
!pip install datasets

In [4]:
!pip install scikit-learn

In [5]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

2024-08-29 17:15:02.598631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 17:15:02.616813: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 17:15:02.622360: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 17:15:02.637714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 17:15:03.652663: W tensorflow/compiler/tf2

In [6]:
@dataclass
class Config:
    gemma_dir = 'unsloth/gemma-2-9b-it-bnb-4bit'
    lora_dir = 'gemma002/checkpoint-417'
    max_length = 512
    batch_size = 24#global_batch_size
    device = torch.device("cuda")    

cfg = Config()

In [7]:
# テストデータの読み込み
test = pd.read_csv("../test.csv")

In [8]:
from typing import Any, Dict, List

def process_text(text: Any) -> str:
    if text is None:
        return ""
    if isinstance(text, str):
        try:
            return " ".join(eval(text, {"null": ""}))
        except:
            return str(text)
    else:
        return str(text)

def tokenize(
    tokenizer, Clothing_ID, Age, Title, Review_Text, Positive_Feedback_Count, max_length=cfg.max_length):

    Clothing_ID = ["<Clothing ID>: " + process_text(t) for t in Clothing_ID]
    Age = ["\n\n<Age>: " + process_text(t) for t in Age]
    Title = ["\n\n<Title>: " + process_text(t) for t in Title]
    Review_Text = ["\n\n<Review Text>: " + process_text(t) for t in Review_Text]
    Positive_Feedback_Count = ["\n\n<Positive Feedback Count>: " + process_text(t) for t in Positive_Feedback_Count]

    texts = [c + a + t + r + p for c, a, t, r, p in zip(Clothing_ID, Age, Title, Review_Text, Positive_Feedback_Count)]
    tokenized = tokenizer(texts, max_length=max_length, truncation=True, padding=False)

    input_ids = tokenized.input_ids
    attention_mask = tokenized.attention_mask

    return input_ids, attention_mask

# データの処理
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["Clothing ID"], test["Age"], test["Title"], test["Review Text"], test["Positive Feedback Count"])
#data["length"] = data["input_ids"].apply(len)

In [9]:
data

,input_ids,attention_mask
0,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...
11150,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11151,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11152,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11153,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
data["input_ids"].value_counts()

input_ids
[2, 235322, 119323, 4781, 78880, 235248, 235284, 235284, 235318, 109, 235322, 11345, 78880, 235248, 235304, 235284, 109, 235322, 4883, 78880, 16769, 109, 235322, 9037, 4820, 78880, 16769, 109, 235322, 35202, 28770, 3522, 78880, 235248, 235276, 1]                                                                                                                                                                                                                                                                                                                                                                                           4
[2, 235322, 119323, 4781, 78880, 235248, 235321, 235304, 109, 235322, 11345, 78880, 235248, 235304, 235308, 109, 235322, 4883, 78880, 16769, 109, 235322, 9037, 4820, 78880, 16769, 109, 235322, 35202, 28770, 3522, 78880, 235248, 235276, 1]                                                                                                                             

In [11]:
test

,Clothing ID,Age,Title,Review Text,Positive Feedback Count
0,0,32,So happy i bought this skirt!,"I love this skirt. it does run a little small,...",0
1,0,34,Runs small,"Beautiful patterns and colors, but it sits ver...",0
2,0,37,Love the comfort of thi skirt,It is easily paired with a nice tip for work a...,0
3,0,39,Way too small,"This is a beautiful skirt, but runs very small...",10
4,0,39,NaN,I usually wear a size 8/10 and i needed a larg...,0
...,...,...,...,...,...
11150,232,53,More beautiful in reality,I purchased this dress on a whim. it is easy t...,0
11151,232,58,Perfect dress!,This dress is very flattering on and can easil...,0
11152,232,60,Perfect dress,This is the most perfect dress i've ever bough...,0
11153,232,62,Really great!,This is a beautiful dress! not too interested ...,5


In [12]:
print(data["input_ids"][0])

[2, 235322, 119323, 4781, 78880, 235248, 235276, 109, 235322, 11345, 78880, 235248, 235304, 235284, 109, 235322, 4883, 78880, 1704, 4915, 496, 8989, 736, 23210, 235341, 109, 235322, 9037, 4820, 78880, 590, 2182, 736, 23210, 235265, 665, 1721, 2060, 476, 2356, 2301, 235269, 496, 11538, 671, 3437, 2301, 578, 665, 603, 11818, 696, 573, 23663, 235269, 901, 2076, 3598, 235265, 496, 791, 4956, 978, 75629, 611, 736, 23210, 1178, 7669, 4341, 496, 791, 14987, 235265, 496, 235303, 524, 1125, 11892, 696, 573, 38701, 4659, 731, 1461, 14926, 682, 1368, 1683, 984, 1154, 665, 235265, 496, 1170, 1160, 675, 5982, 235269, 578, 984, 832, 2182, 665, 685, 1578, 235265, 109, 235322, 35202, 28770, 3522, 78880, 235248, 235276, 1]


In [13]:
print(data["input_ids"][1])

[2, 235322, 119323, 4781, 78880, 235248, 235276, 109, 235322, 11345, 78880, 235248, 235304, 235310, 109, 235322, 4883, 78880, 80963, 2301, 109, 235322, 9037, 4820, 78880, 25111, 12136, 578, 9276, 235269, 901, 665, 27505, 1508, 1536, 578, 10140, 2301, 235265, 496, 235303, 235262, 16342, 476, 2395, 235248, 235310, 578, 2371, 573, 2301, 235265, 143682, 832, 573, 1703, 235269, 901, 611, 573, 68038, 2857, 235265, 573, 23210, 27505, 712, 1536, 674, 573, 9662, 5163, 1706, 729, 476, 3298, 36919, 235265, 1582, 476, 21353, 1861, 665, 2277, 603, 4964, 578, 4713, 1775, 611, 573, 2091, 4485, 1861, 1284, 235303, 235256, 11967, 476, 16716, 577, 14338, 1368, 1536, 665, 37945, 578, 122291, 921, 235265, 109, 235322, 35202, 28770, 3522, 78880, 235248, 235276, 1]


In [14]:
print(tokenizer.decode(data["input_ids"][0]))

<bos><Clothing ID>: 0

<Age>: 32

<Title>: So happy i bought this skirt!

<Review Text>: I love this skirt. it does run a little small, i ordered an extra small and it is tight at the waist, but still works. i have received more compliments on this skirt than maybe anything i have owned. i've been stopped at the grocery store by people telling me how much they like it. i also work with kids, and they all love it as well.

<Positive Feedback Count>: 0<eos>


In [15]:
print(tokenizer.decode(data["input_ids"][1]))

<bos><Clothing ID>: 0

<Age>: 34

<Title>: Runs small

<Review Text>: Beautiful patterns and colors, but it sits very high and runs small. i'm normally a size 4 and got the small. zipped all the way, but on the snug side. the skirt sits so high that the fabric coming down was a bit awkward. such a shame because it really is beautiful and looks great on the model probably because she's wearing a belt to hide how high it rides and flares out.

<Positive Feedback Count>: 0<eos>


In [16]:
print(tokenizer.decode(data["input_ids"][11154]))

<bos><Clothing ID>: 232

<Age>: 63

<Title>: Love this dress!

<Review Text>: This dress is absolutely darling! it is light as a feather and floats on the body. the fabric looks almost handpainted and fits like a dream. i disagree with the previous poster and i think it looks very rich. super comfortable and i know i'll get lots of wear out of it. you can dress it up or dress it down. love it!

<Positive Feedback Count>: 2<eos>


In [17]:
print(tokenizer.decode(data["input_ids"][11154]))

<bos><Clothing ID>: 232

<Age>: 63

<Title>: Love this dress!

<Review Text>: This dress is absolutely darling! it is light as a feather and floats on the body. the fabric looks almost handpainted and fits like a dream. i disagree with the previous poster and i think it looks very rich. super comfortable and i know i'll get lots of wear out of it. you can dress it up or dress it down. love it!

<Positive Feedback Count>: 2<eos>


In [18]:
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)

In [20]:
from tqdm import tqdm

@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    target = []

    # tqdmを使用してプログレスバーを表示
    progress_bar = tqdm(range(0, len(df), batch_size), desc="Inference")
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        target.extend(proba[:, 1].tolist())

        # オプション: プログレスバーの説明を更新して現在の進捗状況を表示
        progress_bar.set_postfix({"Processed": f"{end_idx}/{len(df)}"})
        
    
    df["target"] = target
    
    return df

In [21]:
"""
st = time.time()
results = inference(df=data, model=model_0, device=device_0)

print(f"elapsed time: {time.time() - st}")
"""

original_index = data.index.copy()
results = inference(df=data.reset_index(drop=True), model=model_0, device=device_0)
results = results.set_index(original_index)

Inference:   0%|          | 0/465 [11:17<?, ?it/s, Processed=11155/11155]


In [22]:
results

,input_ids,attention_mask,target
0,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998723
1,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.627281
2,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.997973
3,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.043773
4,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.996031
...,...,...,...
11150,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998364
11151,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.999370
11152,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.999193
11153,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998374


In [23]:
print(tokenizer.decode(results["input_ids"][7863]))

<bos><Clothing ID>: 165

<Age>: 57

<Title>: Cool summer skirt.

<Review Text>: 1. i'm 5'5" tall, 145 lbs, and 38-32-40. i wear a size 30 in hyphen chinos and they are loose. i could get in a size 10, but i felt the stretchiness made it look like it was pulling slightly. i decided to get the size 12 even though it was a hair loose.

2. the skirt hem is right above my knees. it's not as short as it appears on the model.

3. the back pockets are sewn shut, so they won't sag.

4. my only complaint is that the hook and eye at the waist pulls away funny, but i'm going to t

<Positive Feedback Count>: 10<eos>


In [24]:
print(tokenizer.decode(data["input_ids"][7863]))

<bos><Clothing ID>: 165

<Age>: 57

<Title>: Cool summer skirt.

<Review Text>: 1. i'm 5'5" tall, 145 lbs, and 38-32-40. i wear a size 30 in hyphen chinos and they are loose. i could get in a size 10, but i felt the stretchiness made it look like it was pulling slightly. i decided to get the size 12 even though it was a hair loose.

2. the skirt hem is right above my knees. it's not as short as it appears on the model.

3. the back pockets are sewn shut, so they won't sag.

4. my only complaint is that the hook and eye at the waist pulls away funny, but i'm going to t

<Positive Feedback Count>: 10<eos>


In [25]:
#results.reset_index(drop=True)

In [26]:
test.iloc[11150]["Review Text"]

'I purchased this dress on a whim. it is easy to wear and very comfortable.  the empire waist is flattering. however, the dress runs large and the lower half is a-line. the neckline is very flattering and you feel put together when wearing this dress.\r\ni am happy to have it in my wardrobe. a go-to piece for years to come.'

In [27]:
sample_submission = pd.read_csv("../sample_submission.csv")
sample_submission

,target
0,0.488567
1,0.315813
2,0.764448
3,0.980691
4,0.406762
...,...
11150,0.922096
11151,0.692480
11152,0.236563
11153,0.267781


In [28]:
results.reset_index(drop=True)

,input_ids,attention_mask,target
0,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998723
1,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.627281
2,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.997973
3,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.043773
4,"[2, 235322, 119323, 4781, 78880, 235248, 23527...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.996031
...,...,...,...
11150,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998364
11151,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.999370
11152,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.999193
11153,"[2, 235322, 119323, 4781, 78880, 235248, 23528...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.998374


In [29]:
y_pred = results.reset_index(drop=True)["target"]

In [30]:
sample_submission['target'] = y_pred
sample_submission

,target
0,0.998723
1,0.627281
2,0.997973
3,0.043773
4,0.996031
...,...
11150,0.998364
11151,0.999370
11152,0.999193
11153,0.998374


In [31]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6))

vmax = .02
bins = np.linspace(0, 1, 100)
ax.hist(sample_submission['target'], bins=bins, density=True, alpha=.5, label="Test")
ax.grid()
ax.legend()
# ax.set_title("テストと学習時の予測傾向差分")

fig.tight_layout()

In [32]:
sample_submission

,target
0,0.998723
1,0.627281
2,0.997973
3,0.043773
4,0.996031
...,...
11150,0.998364
11151,0.999370
11152,0.999193
11153,0.998374


In [33]:
assert sample_submission["target"].equals(results.reset_index(drop=True)["target"])

In [34]:
sample_submission.to_csv('gemma_ver2_infer.csv', index=False)